# Práctica de Periodismo de Datos: Web Scraping de Titulares y Análisis de Temas

Esta práctica guía a los alumnos a extraer titulares de un sitio de noticias, limpiar el texto y realizar un análisis de frecuencia para identificar las tendencias informativas del día o de una sección específica.

**Objetivos de la Práctica**

1. Web Scraping Estándar: Utilizar requests y BeautifulSoup para extraer titulares y resúmenes de noticias.

2. Limpieza de Texto Avanzada: Procesar el texto eliminando stopwords (palabras comunes) para preparar el análisis.

3. Análisis de Frecuencia: Determinar las palabras clave más recurrentes para identificar los temas de mayor cobertura.

**Requisitos e Instalación**

In [30]:
pip install pandas requests beautifulsoup4

Note: you may need to restart the kernel to use updated packages.


Could not find platform independent libraries <prefix>


## OBJETIVO:

1. Web Scraping de Titulares

El alumno debe elegir una sección de un periódico digital pequeño o un blog de noticias que cargue todas las noticias al inicio sin necesidad de scroll dinámico. Esto asegura que requests pueda descargar todo el contenido relevante.

Tarea 1: Extracción de Titulares y Resúmenes
El alumno debe inspeccionar la página para encontrar el contenedor de cada noticia y las etiquetas específicas de los titulares y resúmenes.

Ejemplo: https://canfali.com/

In [31]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
from collections import Counter

# URL del sitio web a scrapear
url = "https://www.informacion.es/"

# Realizar la petición HTTP
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

response = requests.get(url, headers=headers)
response.encoding = 'utf-8'

# Verificar que la petición fue exitosa
if response.status_code == 200:
    print(f"Conexión exitosa al sitio web (código {response.status_code})")
else:
    print(f"Error en la conexión (código {response.status_code})")

# Parsear el HTML con BeautifulSoup
soup = BeautifulSoup(response.content, 'html.parser')

# Listas para almacenar los datos
titulares = []
resumenes = []
urls = []

# Encontrar todos los artículos
articulos = soup.find_all('article')

print(f"\nSe encontraron {len(articulos)} artículos")

# Extraer titulares, resúmenes y URLs
for articulo in articulos:
    # Buscar el titular
    titular = articulo.find(['h1', 'h2', 'h3', 'h4'])
    if titular:
        titulo_texto = titular.get_text(strip=True)
        
        # Buscar la URL (normalmente está en un enlace <a> dentro del titular o del artículo)
        enlace = titular.find('a') if titular.find('a') else articulo.find('a')
        if enlace and enlace.get('href'):
            url_noticia = enlace.get('href')
            # Si la URL es relativa, agregarle el dominio base
            if url_noticia.startswith('/'):
                url_noticia = url.rstrip('/') + url_noticia
        else:
            url_noticia = ""
        
        titulares.append(titulo_texto)
        urls.append(url_noticia)
        
        # Buscar el resumen
        resumen = articulo.find('p')
        if resumen:
            resumen_texto = resumen.get_text(strip=True)
            resumenes.append(resumen_texto)
        else:
            resumenes.append("")

# Crear DataFrame con los resultados
df_noticias = pd.DataFrame({
    'Titular': titulares,
    'URL': urls,
    'Resumen': resumenes
})

# ELIMINAR DUPLICADOS basándose en el titular
noticias_antes = len(df_noticias)
df_noticias = df_noticias.drop_duplicates(subset=['Titular'], keep='first')
df_noticias = df_noticias.reset_index(drop=True)
noticias_despues = len(df_noticias)

print(f"\nNoticias extraídas: {noticias_antes}")
print(f"Noticias duplicadas eliminadas: {noticias_antes - noticias_despues}")
print(f"Noticias únicas: {noticias_despues}\n")

print("=" * 80)
print("PRIMERAS 5 NOTICIAS ÚNICAS:")
print("=" * 80)
for idx, row in df_noticias.head(5).iterrows():
    print(f"{idx+1}. {row['Titular']}")
    print(f"   URL: {row['URL'][:80]}..." if len(row['URL']) > 80 else f"   URL: {row['URL']}")
    print()
print(df_noticias.info())

# Mostrar todas las noticias en tabla
df_noticias

Conexión exitosa al sitio web (código 200)

Se encontraron 111 artículos

Noticias extraídas: 111
Noticias duplicadas eliminadas: 2
Noticias únicas: 109

PRIMERAS 5 NOTICIAS ÚNICAS:
1. "¿Cuándo he mentido yo?": los ocho cambios de versión de Mazón sobre sus cinco horas clave del 29-O
   URL: https://www.informacion.es/comunidad-valenciana/2025/10/28/cambios-version-menti...

2. El aeropuerto Alicante-Elche vuelve a operar tras el cierre por la presencia de un dron
   URL: https://www.informacion.es/economia/2025/10/27/aeropuerto-alicante-elche-cerrado...

3. Un espacio íntimo para el final de la vida
   URL: https://www.informacion.es/alicante/2025/10/27/hospital-de-alicante-sanidad-cuid...

4. Puigdemont rompe con el PSOE: "Podrá ocupar poltronas, pero no podrá gobernar"
   URL: https://www.informacion.es/nacional/2025/10/27/junts-decide-unanimidad-romper-ps...

5. Cristina fue asesinada en Alicante de cinco puñaladas para robarle 250.000 €
   URL: https://www.informacion.es/sucesos/s

,Titular,URL,Resumen
0,"""¿Cuándo he mentido yo?"": los ocho cambios de ...",https://www.informacion.es/comunidad-valencian...,Comunidad Valenciana
1,El aeropuerto Alicante-Elche vuelve a operar t...,https://www.informacion.es/economia/2025/10/27...,Incidente
2,Un espacio íntimo para el final de la vida,https://www.informacion.es/alicante/2025/10/27...,Sanidad
3,"Puigdemont rompe con el PSOE: ""Podrá ocupar po...",https://www.informacion.es/nacional/2025/10/27...,Junts
4,Cristina fue asesinada en Alicante de cinco pu...,https://www.informacion.es/sucesos/sucesos-en-...,Sucesos en Alicante
...,...,...,...
104,Así puedes suscribirte a Movistar Plus+ por me...,https://www.elperiodico.com/es/shopping/2025/1...,ESNCIAL
105,Amazon despedirá a 30.000 trabajadores a parti...,https://www.informacion.es/videos/economia/202...,Economía
106,Detienen a dos personas en Alicante por el ase...,https://www.informacion.es/videos/sucesos/2025...,Sucesos
107,Los detenidos por la muerte de una mujer en Al...,https://www.informacion.es/videos/alicante/202...,Vídeos de Alicante


2. Limpieza de Texto y Eliminación de Stopwords

Para identificar los temas, debemos deshacernos de las palabras comunes (stopwords: 'el', 'la', 'un', 'es', 'de') que, aunque son necesarias para la gramática, no nos dicen nada sobre el tema de la noticia.

Tarea 2: Limpiar y Filtrar el Texto

In [32]:
# Importar librería para stopwords
import nltk
from nltk.corpus import stopwords

# Descargar stopwords en español (solo necesario la primera vez)
try:
    stopwords_es = set(stopwords.words('spanish'))
except:
    nltk.download('stopwords')
    stopwords_es = set(stopwords.words('spanish'))

print(f"Se cargaron {len(stopwords_es)} stopwords en español")

def limpiar_texto(texto):
    """
    Limpia el texto eliminando caracteres especiales, convirtiendo a minúsculas
    y eliminando stopwords
    """
    # Convertir a minúsculas
    texto = texto.lower()
    
    # Eliminar caracteres especiales y números, mantener solo letras y espacios
    texto = re.sub(r'[^a-záéíóúñü\s]', '', texto)
    
    # Dividir en palabras
    palabras = texto.split()
    
    # Filtrar stopwords y palabras muy cortas (menos de 3 caracteres)
    palabras_filtradas = [palabra for palabra in palabras 
                          if palabra not in stopwords_es and len(palabra) >= 3]
    
    return palabras_filtradas

# Combinar todos los titulares y resúmenes para el análisis
texto_completo = ' '.join(df_noticias['Titular'].fillna('') + ' ' + df_noticias['Resumen'].fillna(''))

print("=" * 80)
print("TEXTO ORIGINAL (primeros 500 caracteres):")
print("=" * 80)
print(texto_completo[:500])
print("\n")

# Limpiar el texto
palabras_limpias = limpiar_texto(texto_completo)

print("=" * 80)
print("PALABRAS LIMPIAS (primeras 50):")
print("=" * 80)
print(palabras_limpias[:50])
print("\n")

print(f"Total de palabras antes de la limpieza: {len(texto_completo.split())}")
print(f"Total de palabras después de la limpieza: {len(palabras_limpias)}")
print(f"Palabras únicas: {len(set(palabras_limpias))}")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ST09\AppData\Roaming\nltk_data...


Se cargaron 313 stopwords en español
TEXTO ORIGINAL (primeros 500 caracteres):
"¿Cuándo he mentido yo?": los ocho cambios de versión de Mazón sobre sus cinco horas clave del 29-O Comunidad Valenciana El aeropuerto Alicante-Elche vuelve a operar tras el cierre por la presencia de un dron Incidente Un espacio íntimo para el final de la vida Sanidad Puigdemont rompe con el PSOE: "Podrá ocupar poltronas, pero no podrá gobernar" Junts Cristina fue asesinada en Alicante de cinco puñaladas para robarle 250.000 € Sucesos en Alicante Comunidad Valenciana, Murcia y Andalucía hacen f


PALABRAS LIMPIAS (primeras 50):
['cuándo', 'mentido', 'ocho', 'cambios', 'versión', 'mazón', 'cinco', 'horas', 'clave', 'comunidad', 'valenciana', 'aeropuerto', 'alicanteelche', 'vuelve', 'operar', 'tras', 'cierre', 'presencia', 'dron', 'incidente', 'espacio', 'íntimo', 'final', 'vida', 'sanidad', 'puigdemont', 'rompe', 'psoe', 'podrá', 'ocupar', 'poltronas', 'podrá', 'gobernar', 'junts', 'cristina', 'asesinada', '

[nltk_data]   Unzipping corpora\stopwords.zip.


3. Análisis de Frecuencia de Temas

Contando la frecuencia de las palabras clave restantes, podemos cuantificar qué términos son los más importantes en el conjunto de noticias, revelando las tendencias de cobertura informativa.

Tarea 3: Conteo y Análisis de las Palabras Clave

In [33]:
# Contar la frecuencia de cada palabra
contador_palabras = Counter(palabras_limpias)

# Obtener las 20 palabras más comunes
palabras_frecuentes = contador_palabras.most_common(20)

# Crear DataFrame con los resultados
df_frecuencias = pd.DataFrame(palabras_frecuentes, columns=['Palabra', 'Frecuencia'])

print("=" * 80)
print("ANÁLISIS DE FRECUENCIA - TOP 20 PALABRAS MÁS COMUNES")
print("=" * 80)
print(df_frecuencias.to_string(index=False))

print("\n")


# Resumen final
print("=" * 80)
print(f"RESUMEN ESTADÍSTICO")
print("=" * 80)
print(f"Total de noticias analizadas: {len(df_noticias)}")
print(f"Total de palabras procesadas: {len(palabras_limpias)}")
print(f"Vocabulario único: {len(set(palabras_limpias))} palabras")
print(f"Palabra más frecuente: '{df_frecuencias.iloc[0]['Palabra']}' ({df_frecuencias.iloc[0]['Frecuencia']} apariciones)")
print("=" * 80)

ANÁLISIS DE FRECUENCIA - TOP 20 PALABRAS MÁS COMUNES
   Palabra  Frecuencia
  alicante          34
     elche          11
  economía          10
       año           8
      años           8
  sociedad           8
      baja           6
  hércules           6
     diego           5
    quiles           4
    último           4
       así           4
      dana           4
       san           4
       día           4
       dos           4
  millones           4
    madrid           4
torrevieja           4
     mujer           4


RESUMEN ESTADÍSTICO
Total de noticias analizadas: 109
Total de palabras procesadas: 968
Vocabulario único: 739 palabras
Palabra más frecuente: 'alicante' (34 apariciones)
